## step 1: go to the right folder

In [7]:
import os
%pwd

'/Users/krystalli/Documents/End-to-End-Wine-Quality-Prediction'

In [2]:
os.chdir('../')
%pwd

'/Users/krystalli/Documents/End-to-End-Wine-Quality-Prediction'

## step 2: update config.yaml

## step 3: create entity

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str

## step 4: create configuration manager

In [9]:
from wine_quality_prediction_project.constants import *
from wine_quality_prediction_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name
        )
        
        return model_evaluation_config

## Step 5: update components

In [12]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import joblib # to load the model
from pathlib import Path
from urllib.parse import urlparse 
from wine_quality_prediction_project.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual_values, predicted_values):
        rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))
        mae = mean_absolute_error(actual_values, predicted_values)
        r2 = r2_score(actual_values, predicted_values)
        return rmse, mae, r2
    
    def save_results(self):
        # load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        # get test x and test y
        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]
        
        # get predictions
        predictions = model.predict(test_x)
        
        # get metrics
        rmse, mae, r2 = self.eval_metrics(test_y, predictions)
        
        # save metrics
        result = {
            "rmse": rmse,
            "mae": mae,
            "r2": r2
        }
        save_json(path=Path(self.config.metric_file_name), data=result) # in utils.common has been defined ("save_json")
        
        
        

# step 6: update pipeline

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2024-11-20 21:11:17,734: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-20 21:11:17,736: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-20 21:11:17,737: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-20 21:11:17,738: INFO: common: created directory at: artifacts]
[2024-11-20 21:11:17,739: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-20 21:11:17,806: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
